![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FRetrieval&file=Retrieval+-+Local+With+Numpy.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Local%20With%20Numpy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FRetrieval%2FRetrieval%2520-%2520Local%2520With%2520Numpy.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Local%20With%20Numpy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Local%20With%20Numpy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Retrieval - Local With Numpy

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [2]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [76]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('numpy', 'numpy'),
    ('sklearn', 'scikit-learn'),
    ('psutil', 'psutil'),
    ('GPUtil', 'GPUtil')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'retrieval-numpy'

Packages

In [87]:
import os, json, sys, time

import numpy as np
import sklearn.cluster
import psutil
import GPUtil

# Vertex AI
from google.cloud import aiplatform
import vertexai.language_models # for embeddings API
import vertexai.generative_models # for Gemini Models

In [9]:
aiplatform.__version__

'1.69.0'

Clients

In [10]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an [example of processing a PDF with the Document AI Layout Parser](../Chunking/Process%20Documents%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [13]:
local_dir = '../Embeddings/files/embeddings-api'

In [14]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [17]:
with open(local_dir+'/chunk-embeddings.jsonl', 'r') as f:
    chunks = [json.loads(line) for line in f]

### Review A Chunk

In [18]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [19]:
chunks[0]['instance']['chunk_id']

'c2'

In [20]:
print(chunks[0]['instance']['content'])

# OFFICIAL BASEBALL RULES

## Official Baseball Rules 2023 Edition

### JOINT COMPETITION COMMITTEE

|-|-|-|
| Bill DeWitt | Whit Merrifield | Austin Slater |
| Jack Flaherty | Bill Miller | John Stanton, Chair |
| Tyler Glasnow | Dick Monfort | Tom Werner |
| Greg Johnson | Mark Shapiro |  |

Committee Secretary Paul V. Mifsud, Jr. Copyright © 2023 by the Office of the Commissioner of Baseball


In [21]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.008681542240083218,
 0.06999468058347702,
 0.003673204220831394,
 0.019888797774910927,
 0.016285404562950134,
 0.035664502531290054,
 0.06200747936964035,
 0.05597030743956566,
 0.0034793149679899216,
 -0.024485772475600243]

### Prepare Chunk Structure

Make a dictionary for each lookup of chunk content by chunk id:

In [22]:
content_chunks = {}
for chunk in chunks:
    content_chunks[chunk['instance']['chunk_id']] = chunk['instance']['content']

In [23]:
content_chunks['c1']

'# OFFICIAL BASEBALL RULES\n\n2023 Edition TM TM'

---
## Simple Retrieval Augmented Generation (RAG): Local With Numpy!

Embeddings can be used with math to measure similarity.  For deeper details into this checkout the companion workflow here: [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb).  Retrieval systems handle the storage and math of similarity as a service.  For an overview of Google Cloud based solutions for retrieval check out [this companion series](../Retrieval/readme.md).

The content below motivates retrieval with the embeddings that accompany the text chunks using a local vector database with brute force matching using [Numpy](https://numpy.org/)!

### Vector DB With Numpy

In [30]:
vector_db = [
    [
        chunk['instance']['chunk_id'],
        chunk['predictions'][0]['embeddings']['values'],
    ]
    for chunk in chunks
]
vector_index = np.array([row[1] for row in vector_db])

In [31]:
len(vector_db)

867

In [32]:
vector_index.shape

(867, 768)

### Models: Embeddings, Generation

Connect to models for text embeddings and text generation:

Learn more about these model APIs:
- [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)
- [Vertex AI Gemini API](../Generate/Vertex%20AI%20Gemini%20API.ipynb)

In [33]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')
llm = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-001")

Define a question that is the start of our prompt to the LLM:

In [34]:
question = "What are the dimensions of a base?"

Get an ungrounded response to the question with the LLM:

In [35]:
print(llm.generate_content(question).text)

The term "base" can have different meanings depending on the context. Please clarify what you mean by "base". Here are some possible interpretations:

**In geometry:**

* **Base of a triangle:**  A side of a triangle, often the one perpendicular to the altitude (height).
* **Base of a rectangle or parallelogram:**  Any one of the sides of the shape.
* **Base of a pyramid or cone:**  The flat face on which the shape rests.
* **Base of a cylinder:**  The circular face on which the cylinder rests.

**In other contexts:**

* **Base of a number system:** The number of unique digits used in a number system (e.g., base-10 for decimal numbers).
* **Base of a chemical compound:** The atom or group of atoms that forms the core structure of the molecule. 

**Please provide more information about the context of the "base" you are referring to, and I can help you determine its dimensions.** 



Get an embedding for the question to use in retrieval:

In [36]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.026682045310735703,
 0.011593513190746307,
 0.028523651883006096,
 -0.0017065361607819796,
 0.01946176588535309,
 0.0031198114156723022,
 0.07915323227643967,
 -0.005078596994280815,
 -0.006295712199062109,
 0.04943541809916496]

### Retrieval: Matching With Numpy

Use dot product to calculate similarity and find matches for a query embedding.  Why dot product?  Check out the companion workflow: [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb)

> **NOTE:**  This will calculate the similarity for all embeddings vectors stored in the local vector db which is just a Numpy array here.  This is very fast because there are <200 embeddings vectors.  As this scales it would be better to consider a solution that searches a subset of embeddings.  More details on retrieval solutions can be found in [Retrieval](../Retrieval/readme.md).

In [105]:
similarity = np.dot(question_embedding, vector_index.T)
matches = np.argsort(similarity)[-5:].tolist()
matches = [(match, similarity[match]) for match in matches]
matches

[(26, 0.5033481946111171),
 (40, 0.5126844935129918),
 (836, 0.5244194362041271),
 (36, 0.5724333016720691),
 (38, 0.5843799337008113)]

In [101]:
for m, match in enumerate(matches):
    print(f"Match {m+1} ({match[1]:.2f}) is chunk {vector_db[match[0]][0]}:\n{content_chunks[vector_db[match[0]][0]]}\n###################################################")

Match 1 (0.50) is chunk c31:
# 2.00-THE PLAYING FIELD

## 2.01 Layout of the Field

When location of home base is determined, with a steel tape measure 127 feet, 3\frac{3}{8} inches in desired direction to establish second base. From home base, measure 90 feet toward first base; from second base, measure 90 feet toward first base; the intersection of these lines establishes first base. From home base, measure 90 feet toward third base; from second base, measure 90 feet toward third base; the intersection of these lines establishes third base.
###################################################
Match 2 (0.51) is chunk c40:
# Rule 2.03 to 2.05

## 2.03 The Bases

First, second and third bases shall be marked by white canvas or rubber-covered bags, securely attached to the ground as indicated in Diagram 2. The first and third base bags shall be entirely within the infield. The second base bag shall be centered on second base. The bags shall be 18 inches square, not less than three nor mor

### Generation: Q&A With Gemini Grounded With RAG

Provide the matched chunks of text along with the question as a prompt to a generative model for a grounded answer.

#### Prompt Building Function

Use the matching chunks as context for the prompt:

In [102]:
def get_prompt(question, top_n = 5):
    # get embedding for question
    question_embedding = embedder.get_embeddings([question])[0].values
    # get top_n matches:
    similarity = np.dot(question_embedding, vector_index.T)
    matches = np.argsort(similarity)[-top_n:].tolist()
    matches = [[match, similarity[match]] for match in matches]
    # construct prompt:
    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n{question}'
    
    return matches, prompt

In [103]:
matches, prompt = get_prompt(question) 
print(prompt)

Context 1:
# 2.00-THE PLAYING FIELD

## 2.01 Layout of the Field

When location of home base is determined, with a steel tape measure 127 feet, 3\frac{3}{8} inches in desired direction to establish second base. From home base, measure 90 feet toward first base; from second base, measure 90 feet toward first base; the intersection of these lines establishes first base. From home base, measure 90 feet toward third base; from second base, measure 90 feet toward third base; the intersection of these lines establishes third base.

Context 2:
# Rule 2.03 to 2.05

## 2.03 The Bases

First, second and third bases shall be marked by white canvas or rubber-covered bags, securely attached to the ground as indicated in Diagram 2. The first and third base bags shall be entirely within the infield. The second base bag shall be centered on second base. The bags shall be 18 inches square, not less than three nor more than five inches thick, and filled with soft material.

Context 3:
# APPENDICES

## A

### Grounded Generation

In [104]:
answer = llm.generate_content(prompt).text
print(answer)

According to Context 3, the bases are **18 inches square**, with a thickness of **between 3 and 5 inches**. 



---
## Profiling Performance



### Size Of Objects

The design above involves three objects:
- `vector_db` - a Python list of list objects that each contain a chunk_id and the embedding vector for the chunk
- `vector_index` - a numpy array of rows for each embedding vector
- `content_chunks` - a Python dict that has keys for each chunk_id and values are the text of the chunk

These are used by finding the index of matching embeddings from the `vector_index` and then looking up the cooresponding chunk_id in `vector_db` before finally retrieving the text of the chunk from `content_chunks`.

#### Object: `vector_db`

In [43]:
type(vector_db)

list

In [47]:
len(vector_db)

867

In [56]:
# size in bytes
sys.getsizeof(vector_db)

7832

In [57]:
# size in megabytes
sys.getsizeof(vector_db)/ (1024*1024)

0.00746917724609375

#### Object: `vector_index`

In [51]:
type(vector_index)

numpy.ndarray

In [52]:
vector_index.shape

(867, 768)

In [53]:
# size in bytes
sys.getsizeof(vector_index)

5326976

In [58]:
# size in megabytes
sys.getsizeof(vector_index)/ (1024*1024)

5.0802001953125

#### Object: `content_chunks`

In [59]:
type(content_chunks)

dict

In [60]:
len(content_chunks)

867

In [61]:
# size in bytes
sys.getsizeof(content_chunks)

36960

In [62]:
# size in megabytes
sys.getsizeof(content_chunks)/ (1024*1024)

0.035247802734375

### Local Compute Environment

In [82]:
# Get CPU count
cpu_count = psutil.cpu_count(logical=True)  # Includes logical cores (hyperthreading)
print(f"CPU Count: {cpu_count}")

# Get CPU frequency
cpu_freq = psutil.cpu_freq()
print(f"CPU Frequency: {cpu_freq.current} MHz")

CPU Count: 4
CPU Frequency: 2199.998 MHz


In [80]:
mem = psutil.virtual_memory()
print(f"Total Memory: {mem.total / (1024**3):.2f} GB")
print(f"Used Memory: {mem.used / (1024**3):.2f} GB")
print(f"Available Memory: {mem.available / (1024**3):.2f} GB")
print(f"Memory Percentage Used: {mem.percent}%")

Total Memory: 31.36 GB
Used Memory: 14.50 GB
Available Memory: 16.42 GB
Memory Percentage Used: 47.6%


In [81]:
# Get all available GPUs
gpus = GPUtil.getGPUs()

if gpus:
    for gpu in gpus:
        print(f"GPU Name: {gpu.name}")
        print(f"GPU Memory Total: {gpu.memoryTotal} MB")
        print(f"GPU Memory Used: {gpu.memoryUsed} MB")
        print(f"GPU Memory Free: {gpu.memoryFree} MB")
        print(f"GPU Load: {gpu.load*100}%")
else:
    print("No GPUs found.")

No GPUs found.


### Timing Sequential Operations

Get the timing for different sequential matching requests loads: 1, 10, 100, 1000, 10000, 100000, ...

Break this down by the tasks:
- Get Embedding Vector For Question
- Get Matching Chunks
- Construct Prompt From Matches

#### Get Embedding Vector For Question: API

This test sequential request.  The Text Embeddings API has many option for asynchronous and multi-instance request that could also be used for efficiency.  See more in [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

In [70]:
embed_time = []
for x in range(4):
    n = 10**x
    start_time = time.time()
    for i in range(n):
        # get embedding for question
        question_embedding = embedder.get_embeddings([question])[0].values
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for n = {n}: {execution_time:.6f} seconds")
    embed_time.append(execution_time)

Execution time for n = 1: 0.083227 seconds
Execution time for n = 10: 0.887779 seconds
Execution time for n = 100: 5.782034 seconds
Execution time for n = 1000: 58.156775 seconds


#### Get Matching Chunks: Python + Numpy

In [106]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

match_time = []
for x in range(4):
    n = 10**x
    start_time = time.time()
    for i in range(n):
        # get top_n matches:
        top_n = 10
        similarity = np.dot(question_embedding, vector_index.T)
        matches = np.argsort(similarity)[-top_n:].tolist()
        matches = [[match, similarity[match]] for match in matches]
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for n = {n}: {execution_time:.6f} seconds")
    match_time.append(execution_time)

Execution time for n = 1: 0.000586 seconds
Execution time for n = 10: 0.004576 seconds
Execution time for n = 100: 0.041530 seconds
Execution time for n = 1000: 0.409031 seconds


#### Construct Prompt Form Matches: Python

In [108]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

# get top_n matches:
top_n = 10
similarity = np.dot(question_embedding, vector_index.T)
matches = np.argsort(similarity)[-top_n:].tolist()
matches = [[match, similarity[match]] for match in matches]

prompt_time = []
for x in range(4):
    n = 10**x
    start_time = time.time()
    for i in range(n):
        # construct prompt:
        prompt = ''
        for m, match in enumerate(matches):
            prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
        prompt += f'Answer the following question using the provided contexts:\n{question}'
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for n = {n}: {execution_time:.6f} seconds")
    prompt_time.append(execution_time)

Execution time for n = 1: 0.000063 seconds
Execution time for n = 10: 0.000231 seconds
Execution time for n = 100: 0.002236 seconds
Execution time for n = 1000: 0.022976 seconds


#### Combined Local Process of Matching, Prompt Construction:

Leave out the embeddings requests where are API wait time to the local processing.

In [168]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

combined_time = []
for x in range(6):
    n = 10**x
    start_time = time.time()
    for i in range(n):

        # get top_n matches:
        top_n = 10
        similarity = np.dot(question_embedding, vector_index.T)
        matches = np.argsort(similarity)[-top_n:].tolist()
        matches = [[match, similarity[match]] for match in matches]
        
        # construct prompt:
        prompt = ''
        for m, match in enumerate(matches):
            prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
        prompt += f'Answer the following question using the provided contexts:\n{question}'
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for n = {n}: {execution_time:.6f} seconds")
    combined_time.append(execution_time)

Execution time for n = 1: 0.001533 seconds
Execution time for n = 10: 0.017490 seconds
Execution time for n = 100: 0.126925 seconds
Execution time for n = 1000: 0.491893 seconds
Execution time for n = 10000: 4.189425 seconds
Execution time for n = 100000: 44.745094 seconds


### Profile Sequential Operations Timing

Now collect the individual timings for local operations and review the profile of timing:

In [169]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

combined_time_profile = []

n = 10000

for i in range(n):
    start_time = time.time()
    
    # get top_n matches:
    top_n = 10
    similarity = np.dot(question_embedding, vector_index.T)
    matches = np.argsort(similarity)[-top_n:].tolist()
    matches = [[match, similarity[match]] for match in matches]

    # construct prompt:
    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n{question}'
    
    end_time = time.time()
    execution_time = end_time - start_time
    combined_time_profile.append(execution_time)

In [170]:
# Total time for all requests
total_time = sum(combined_time_profile)
print(f"Total time for all requests: {total_time:.6f} seconds")

# Average time per request
average_time = total_time / len(combined_time_profile)
print(f"Average time per request: {average_time:.6f} seconds")

# Range of time across all requests
time_range = max(combined_time_profile) - min(combined_time_profile)
print(f"Range of time across all requests: {time_range:.6f} seconds")

# 99th percentile of request times
percentile_99 = np.percentile(combined_time_profile, 99)
print(f"99th percentile of request times: {percentile_99:.6f} seconds") 

Total time for all requests: 5.589185 seconds
Average time per request: 0.000559 seconds
Range of time across all requests: 0.016161 seconds
99th percentile of request times: 0.001382 seconds


---
## Approximate Search With IVF using K-Means

The solution above is fast at the current size and scale.  As the number of embeddings increase it could be helpful to search a subset of embeddings for faster responses.  A simple way to extend the brute-force search to a subset is an Inverted File (IVF) index. How?
- Cluster the embeddings into k groups, using [k-means clustering](https://en.wikipedia.org/wiki/K-means_clustering)
- Create an inverted list that assigns embeddings to clusters
- Search by first finding the closest cluster then only searching within those

Here the clustering with k-means is trained with [scikit-learn `sklearn.cluster.KMeans`](https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html).

### Cluster With k-means

In [112]:
k = 100
kmeans = sklearn.cluster.KMeans(n_clusters = k, random_state = 0)
cluster_assignments = kmeans.fit_predict(vector_index)

### Create Inverted Lists

In [113]:
inverted_lists = [[] for _ in range(k)]
for i, cluster_id in enumerate(cluster_assignments):
    inverted_lists[cluster_id].append(i)

In [114]:
len(inverted_lists)

100

In [115]:
inverted_lists[0]

[376,
 457,
 460,
 465,
 470,
 472,
 473,
 474,
 475,
 476,
 477,
 478,
 480,
 481,
 482,
 484,
 485,
 487]

### Search With IVF Index

#### Find Closest Clusters

The center of each cluster is stored:

In [116]:
kmeans.cluster_centers_.shape

(100, 768)

In [119]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

In [125]:
cluster_similarity = np.dot(question_embedding, kmeans.cluster_centers_.T)
nearest_clusters = np.argsort(cluster_similarity)[-10:]

In [126]:
nearest_clusters

array([89, 58, 28,  9, 53, 17, 59, 50, 14, 30])

#### Search Within Clusters

In [133]:
candidate_indicies = [idv for cluster_id in nearest_clusters for idv in inverted_lists[cluster_id]]

#### Top Matches Within Candidate Clusters

In [145]:
candidate_index = vector_index[candidate_indicies]
candidate_similarity = np.dot(question_embedding, candidate_index.T)
ivf_matches = [[candidate_indicies[match], candidate_similarity[match]] for match in np.argsort(candidate_similarity)[-top_n:].tolist()]
ivf_matches

[[27, 0.47031834864450583],
 [838, 0.47357323331558154],
 [840, 0.4883681719341052],
 [837, 0.4904491447830121],
 [29, 0.5006052048144691],
 [26, 0.5033481946111171],
 [40, 0.5126844935129918],
 [836, 0.5244194362041271],
 [36, 0.5724333016720693],
 [38, 0.5843799337008113]]

#### Compare To Top Matches From Brute-Force

In [146]:
top_n = 10
similarity = np.dot(question_embedding, vector_index.T)
matches = np.argsort(similarity)[-top_n:].tolist()
matches = [[match, similarity[match]] for match in matches]
matches

[[27, 0.47031834864450583],
 [838, 0.47357323331558154],
 [840, 0.4883681719341052],
 [837, 0.4904491447830121],
 [29, 0.5006052048144691],
 [26, 0.5033481946111171],
 [40, 0.5126844935129918],
 [836, 0.5244194362041271],
 [36, 0.5724333016720691],
 [38, 0.5843799337008113]]

In [148]:
[i[0] for i in matches] == [i[0] for i in ivf_matches]

True

#### Put Steps Together For Efficiency

In [173]:
nearest_clusters = np.argsort(np.dot(question_embedding, kmeans.cluster_centers_.T))[-top_c:]
candidate_indices = np.concatenate([inverted_lists[cluster_id] for cluster_id in nearest_clusters])
candidate_similarity = np.dot(question_embedding, vector_index[candidate_indices].T)
top_indices = np.argsort(candidate_similarity)[-top_n:]
ivf_matches = [[candidate_indices[i], candidate_similarity[i]] for i in top_indices]
ivf_matches

[[27, 0.47031834864450583],
 [838, 0.47357323331558154],
 [840, 0.4883681719341052],
 [837, 0.4904491447830121],
 [29, 0.5006052048144691],
 [26, 0.5033481946111171],
 [40, 0.5126844935129918],
 [836, 0.5244194362041271],
 [36, 0.5724333016720693],
 [38, 0.5843799337008113]]

### Time Sequential Operations

Similar to the brute-force timing above, calculate the time for various numbers of sequential operations:

#### Combined Local Process of Matching, Prompt Construction:

Leave out the embeddings requests where are API wait time to the local processing.

In [174]:
top_c = 10 # number of clusters to match
top_n = 10 # number of neighbors to match
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

combined_time_ivf = []
for x in range(6):
    n = 10**x
    start_time = time.time()
    for i in range(n):
        # get top_n matches with IVF
        nearest_clusters = np.argsort(np.dot(question_embedding, kmeans.cluster_centers_.T))[-top_c:]
        candidate_indices = np.concatenate([inverted_lists[cluster_id] for cluster_id in nearest_clusters])
        candidate_similarity = np.dot(question_embedding, vector_index[candidate_indices].T)
        top_indices = np.argsort(candidate_similarity)[-top_n:]
        ivf_matches = [[candidate_indices[i], candidate_similarity[i]] for i in top_indices]
        
        # construct prompt:
        prompt = ''
        for m, match in enumerate(ivf_matches):
            prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
        prompt += f'Answer the following question using the provided contexts:\n{question}'        

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for n = {n}: {execution_time:.6f} seconds")
    combined_time_ivf.append(execution_time)

Execution time for n = 1: 0.001008 seconds
Execution time for n = 10: 0.007617 seconds
Execution time for n = 100: 0.065586 seconds
Execution time for n = 1000: 0.651683 seconds
Execution time for n = 10000: 6.369548 seconds
Execution time for n = 100000: 65.594522 seconds


Compare timing:

In [175]:
for i, (t, ivf_t) in enumerate(zip(combined_time, combined_time_ivf)):
    adiff = abs(t-ivf_t)
    if t <= ivf_t:
        print(f"For {10**i} iterations IVF was {100*(adiff/t):.2f}% slower")
    else:
        print(f"For {10**i} iterations: IVF was {100*(adiff/t):.2f}% faster")

For 1 iterations: IVF was 34.25% faster
For 10 iterations: IVF was 56.45% faster
For 100 iterations: IVF was 48.33% faster
For 1000 iterations IVF was 32.48% slower
For 10000 iterations IVF was 52.04% slower
For 100000 iterations IVF was 46.60% slower


### Profile Sequential Operations Timing

Now collect the individual timings for local operations and review the profile of timing:

In [178]:
# get embedding for question
question_embedding = embedder.get_embeddings([question])[0].values

combined_time_profile_ivf = []

n = 10000

for i in range(n):
    start_time = time.time()
    
    top_c = 10 # number of clusters to match
    top_n = 10 # number of neighbors to match
    # get top_n matches with IVF
    nearest_clusters = np.argsort(np.dot(question_embedding, kmeans.cluster_centers_.T))[-top_c:]
    candidate_indices = np.concatenate([inverted_lists[cluster_id] for cluster_id in nearest_clusters])
    candidate_similarity = np.dot(question_embedding, vector_index[candidate_indices].T)
    top_indices = np.argsort(candidate_similarity)[-top_n:]
    ivf_matches = [[candidate_indices[i], candidate_similarity[i]] for i in top_indices]

    # construct prompt:
    prompt = ''
    for m, match in enumerate(ivf_matches):
        prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n{question}'
    
    end_time = time.time()
    execution_time = end_time - start_time
    combined_time_profile_ivf.append(execution_time)

In [179]:
# Total time for all requests
total_time = sum(combined_time_profile_ivf)
print(f"Total time for all requests: {total_time:.6f} seconds")

# Average time per request
average_time = total_time / len(combined_time_profile_ivf)
print(f"Average time per request: {average_time:.6f} seconds")

# Range of time across all requests
time_range = max(combined_time_profile_ivf) - min(combined_time_profile_ivf)
print(f"Range of time across all requests: {time_range:.6f} seconds")

# 99th percentile of request times
percentile_99 = np.percentile(combined_time_profile_ivf, 99)
print(f"99th percentile of request times: {percentile_99:.6f} seconds") 

Total time for all requests: 6.331494 seconds
Average time per request: 0.000633 seconds
Range of time across all requests: 0.022187 seconds
99th percentile of request times: 0.001050 seconds


Compare Timing:

In [198]:
def print_time_stats(label, times):
    """Prints timing statistics for a given list of times."""
    total_time = sum(times)
    average_time = total_time / len(times)
    time_range = max(times) - min(times)
    percentile_99 = np.percentile(times, 99)
    percentile_97 = np.percentile(times, 97)
    percentile_95 = np.percentile(times, 95)
    print(f"----- {label} -----")
    print(f"Total time: {total_time:.6f} seconds")
    print(f"Average time: {average_time:.6f} seconds")
    print(f"Range: {time_range:.6f} seconds")
    print(f"99th percentile: {percentile_99:.6f} seconds")
    print(f"97th percentile: {percentile_97:.6f} seconds")
    print(f"95th percentile: {percentile_95:.6f} seconds")
    return (total_time, average_time, time_range, percentile_99, percentile_97, percentile_95)
    
# Print individual statistics
results_ivf = print_time_stats("IVF", combined_time_profile_ivf)
results_bf = print_time_stats("Brute Force", combined_time_profile)

----- IVF -----
Total time: 6.331494 seconds
Average time: 0.000633 seconds
Range: 0.022187 seconds
99th percentile: 0.001050 seconds
97th percentile: 0.000926 seconds
95th percentile: 0.000889 seconds
----- Brute Force -----
Total time: 5.589185 seconds
Average time: 0.000559 seconds
Range: 0.016161 seconds
99th percentile: 0.001382 seconds
97th percentile: 0.000990 seconds
95th percentile: 0.000945 seconds


In [199]:
metrics = {'1' : 'Total time', '2' : 'Average Time', '3' : 'Range', '4' : '99th percentile', '5':'97th percentile', '6':'95th percentile'}
for i, (bf, ivf) in enumerate(zip(results_bf, results_ivf)):
    adiff = abs(bf-ivf)
    if bf <= ivf:
        print(f"For the '{metrics[str(i+1)]}' IVF was {100*(adiff/bf):.2f}% slower:\n\tbrute force = {bf:.6f}\n\tIVF = {ivf:.6f}")
    else:
        print(f"For the '{metrics[str(i+1)]}' IVF was {100*(adiff/bf):.2f}% faster:\n\tbrute force = {bf:.6f}\n\tIVF = {ivf:.6f}")


For the 'Total time' IVF was 13.28% slower:
	brute force = 5.589185
	IVF = 6.331494
For the 'Average Time' IVF was 13.28% slower:
	brute force = 0.000559
	IVF = 0.000633
For the 'Range' IVF was 37.28% slower:
	brute force = 0.016161
	IVF = 0.022187
For the '99th percentile' IVF was 23.98% faster:
	brute force = 0.001382
	IVF = 0.001050
For the '97th percentile' IVF was 6.43% faster:
	brute force = 0.000990
	IVF = 0.000926
For the '95th percentile' IVF was 5.96% faster:
	brute force = 0.000945
	IVF = 0.000889
